In [1]:
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np
from tqdm.notebook import tqdm as tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, KFold
import pickle
from pymoo.algorithms.soo.nonconvex.pso import PSO, PSOAnimation
from pymoo.optimize import minimize
import matplotlib.pyplot as plt
from pymoo.factory import get_termination
from pymoo.core.callback import Callback
from catboost import CatBoostClassifier
import numpy as np
from pymoo.core.problem import ElementwiseProblem

In [2]:
with open('./fruit.pkl', 'rb') as f:
    x_fruit, y_fruit = pickle.load(f)

In [3]:
iterations_min = 20  #n_estimators n_estimators #INT
learning_rate_min = 0.001  #FLOAT
l2_leaf_reg_min = 0.01  #FLOAT reg_lambda
bagging_temperature_min = 0.001  # float
subsample_min = 1.0   #float 
mvs_reg_min = 0.001  #float 0 - inf
random_strength_min = 0.001  #float 1.0 
use_best_model_min = False  #bool true false
best_model_min_trees_min = 1  #int 
depth_min = 1  #max_depth 8 int
min_data_in_leaf_min = 1  #min_child_samples int
max_leaves_min = 10  #num_leaves int 
one_hot_max_size_min = 1  #int 
rsm_min = 0.001  #colsample_bylevel float (0;1]
fold_permutation_block_min = 1  #int 
leaf_estimation_iterations_min = 1  #int
fold_len_multiplier_min = 2.0  #float
approx_on_full_history_min = False  #bool 
boost_from_average_min = False  #bool 
diffusion_temperature_min = 1000  #float 10000
posterior_sampling_min = False  #bool
allow_const_label_min = False  #bool
penalties_coefficient_min = 1.0  #float
model_shrink_rate_min = 0.001  #float

iterations_max = 100  #n_estimators n_estimators #INT
learning_rate_max = 0.5  #FLOAT
l2_leaf_reg_max = 0.01  #FLOAT reg_lambda
bagging_temperature_max = 10000.0  # float
subsample_max = 2.0  #float 
mvs_reg_max = 1000.0  #float 0 - inf
random_strength_max = 10.0  #float 1.0 
use_best_model_max = True  #bool true false
best_model_min_trees_max = 10  #int 
depth_max = 1  #max_depth 8 int 
min_data_in_leaf_max = 100  #min_child_samples int
max_leaves_max = 10  #num_leaves int 
one_hot_max_size_max = 20  #int 
rsm_max = 1.0   #colsample_bylevel float (0;1]
fold_permutation_block_max = 100  #int 
leaf_estimation_iterations_max = 100  #int
fold_len_multiplier_max = 10.0  #float
approx_on_full_history_max = True  #bool 
boost_from_average_max = True  #bool 
diffusion_temperature_max = 50000  #float 10000
posterior_sampling_max = True  #bool
allow_const_label_max = True  #bool
penalties_coefficient_max = 10.0  #float
model_shrink_rate_max = 1.0  #float

grow_policy = 'Lossguide'

In [4]:
class OptimizeWithAccuracy(ElementwiseProblem):
    def __init__(self):
        xl=np.array([iterations_min,
                    learning_rate_min,
                    l2_leaf_reg_min,
                    bagging_temperature_min,
                    random_strength_min,
                    best_model_min_trees_min,
                    depth_min,
                    min_data_in_leaf_min,
                    max_leaves_min,
                    one_hot_max_size_min,
                    rsm_min,
                    fold_permutation_block_min,
                    leaf_estimation_iterations_min,
                    fold_len_multiplier_min,
                    penalties_coefficient_min,
                    model_shrink_rate_min])
        xu=np.array([iterations_max,
                    learning_rate_max,
                    l2_leaf_reg_max,
                    bagging_temperature_max,
                    random_strength_max,
                    best_model_min_trees_max,
                    depth_max,
                    min_data_in_leaf_max,
                    max_leaves_max,
                    one_hot_max_size_max,
                    rsm_max,
                    fold_permutation_block_max,
                    leaf_estimation_iterations_max,
                    fold_len_multiplier_max,
                    penalties_coefficient_max,
                    model_shrink_rate_max])

        super().__init__(n_var=len(xu), n_obj=1, n_ieq_constr=2, xl=xl, xu=xu)

    def _evaluate(self, x, out, *args, **kwargs):
        model_catboost = CatBoostClassifier(
                                        iterations = int(np.round(x[0])),
                                        learning_rate = x[1],
                                        l2_leaf_reg = x[2],
                                        bagging_temperature = x[3],
                                        random_strength = x[4],
                                        best_model_min_trees = int(np.round(x[5])), 
                                        depth = int(np.round(x[6])), 
                                        min_data_in_leaf = int(np.round(x[7])), 
                                        max_leaves = int(np.round(x[8])), 
                                        one_hot_max_size = int(np.round(x[9])), 
                                        rsm = x[10],
                                        fold_permutation_block = int(np.round(x[11])), 
                                        leaf_estimation_iterations = int(np.round(x[12])), 
                                        fold_len_multiplier = x[13],
                                        approx_on_full_history = False,
                                        penalties_coefficient = x[14],
                                        model_shrink_rate = x[15],
                                        loss_function='MultiClass',
                                        grow_policy = 'Lossguide',
                                        thread_count = -1
                                       )
        
        kfold = KFold(n_splits = 3, shuffle = True)
        
        scores = cross_val_score(model_catboost, x_fruit, y_fruit, cv = kfold, n_jobs=-1)  
        result = scores.mean()     
        #out['F'] = -1 * result
        #out['G'] = -1 * result
        
        if result is None:
            out['F'] = -1e6 # return a large negative number instead of None
        else:
            out['F'] = -1 * result
        
        
problemAccuracy = OptimizeWithAccuracy()

In [5]:
from pymoo.util.display.column import Column
from pymoo.util.display.output import Output


class MyOutput(Output):

    def __init__(self):
        super().__init__()
        self.score = Column("score", width=13)
        self.columns += [self.score]

    def update(self, algorithm):
        super().update(algorithm)
        self.score.set(-np.min(algorithm.pop.get("F")))

In [6]:
def run_accuracy_pso(ITERATIONS = 32, POPULATION = 32):
    algorithm = PSO(max_velocity_rate=0.25,
                    pop_size=POPULATION)
                    
    term = get_termination("n_gen", ITERATIONS)

    res = minimize(problemAccuracy,
                algorithm,
                #seed=SEED,
                save_history=False,
                verbose=True,
                output=MyOutput(),
                termination = term)


    #index_best_individual = np.where(res.pop.get('F') == np.min(res.pop.get('F')))[0][0]
    #score_best_individual = res.pop.get('F')[index_best_individual]
    #parameters_best_individual = res.pop.get('X')[index_best_individual]

    #print(f'Best Accuracy Score {-score_best_individual}')
    #print(f'Model parameters: \n {parameters_best_individual}')
    #return score_best_individual, parameters_best_individual, res

In [7]:
run_accuracy_pso()

C:\Users\Felps\AppData\Local\Temp\ipykernel_7768\141096362.py:5: DeprecationWarning: Call to deprecated function (or staticmethod) get_termination. (Please use `from pymoo.termination import get_termination`)
  term = get_termination("n_gen", ITERATIONS)


n_gen  |  n_eval  |     score    
     1 |       32 |  0.2271683389


c:\Users\Felps\anaconda3\envs\pymoo\lib\site-packages\pymoo\util\misc.py:99: RuntimeWarning: invalid value encountered in divide
  return lambda a, b: np.sqrt((((a - b) / (xu - xl)) ** 2).sum(axis=1))


TypeError: '>' not supported between instances of 'NoneType' and 'NoneType'

In [8]:
import pymoo
print(pymoo.__version__)

0.6.0.1


In [ ]:
from pymoo.optimize import minimize
from pymoo.termination import get_termination
from pymoo.algorithms.soo.nonconvex.ga import GA

algorithm = GA(pop_size=POPULATION)

term = get_termination("n_gen", ITERATIONS)

res = minimize(problem,
               algorithm,
               #seed=SEED,
               save_history=True,
               output=MyOutput(),
               verbose=True,
               termination = term)

NameError: name 'POPULATION' is not defined